In [3]:
from bs4 import BeautifulSoup
from pytube import YouTube
import os
import re
from moviepy.video.io.VideoFileClip import VideoFileClip

c:\Users\mainp\AppData\Local\pypoetry\Cache\virtualenvs\data-scraping-python-zETLVNi3-py3.12\Lib\site-packages\moviepy\config_defaults.py:1: SyntaxWarning: invalid escape sequence '\P'
  """
c:\Users\mainp\AppData\Local\pypoetry\Cache\virtualenvs\data-scraping-python-zETLVNi3-py3.12\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
c:\Users\mainp\AppData\Local\pypoetry\Cache\virtualenvs\data-scraping-python-zETLVNi3-py3.12\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
c:\Users\mainp\AppData\Local\pypoetry\Cache\virtualenvs\data-scraping-python-zETLVNi3-py3.12\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)


In [4]:
# import the html from the youtube.html file
with open('youtube.html', 'r', encoding='utf-8') as f:
    html = f.read()

In [5]:
urls = []
soup = BeautifulSoup(html, 'html.parser')
thumbnails = soup.find_all('a', {'id': 'thumbnail'})
for thumbnail in thumbnails:
    href = thumbnail.get('href')
    if href and '/watch?v=' in href:
        urls.append("https://www.youtube.com" + href)


# remove the duplicates
urls = list(set(urls))
print(len(urls))

# export all the urls
with open('urls.txt', 'w') as f:
    for url in urls:
        f.write(url + '\n')

142


In [6]:
html = open('youtube.html', 'r', encoding='utf-8').read()


# Define the regular expression pattern
pattern = r'"url":\s*"(/watch\?v=[^"]+)"'

# Find all matches in the source code
matches = re.findall(pattern, html)

urls = []

# Append all the extracted video URLs
for match in matches:
    urls.append("https://www.youtube.com" + match)
urls = list(set(urls))
print(len(urls))

# export all the urls
with open('urls.txt', 'w') as f:
    for url in urls:
        f.write(url + '\n')

0


In [7]:
failed = []
for url in urls:
    try:

        yt = YouTube(url)

        # get the best quality video stream

        best_video_stream = yt.streams.get_highest_resolution()
        # Download the video file

        best_video_stream.download()
    except Exception as e:
        print(e)
        failed.append(url)

In [8]:


def convert_mp4_to_mp3(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the input folder
    files = os.listdir(input_folder)

    for file in files:
        # Check if the file is an MP4 file
        if file.endswith(".mp4"):
            input_path = os.path.join(input_folder, file)

            # Generate output file path with the same name but with .mp3 extension
            output_file = os.path.splitext(file)[0] + ".mp3"
            output_path = os.path.join(output_folder, output_file)

            # Convert MP4 to MP3 using moviepy
            video_clip = VideoFileClip(input_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(output_path, codec='mp3')

            # Close the clips
            audio_clip.close()
            video_clip.close()


if __name__ == "__main__":
    input_folder = "./mp4/"
    output_folder = "./mp3/"

    convert_mp4_to_mp3(input_folder, output_folder)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './mp4/'

In [2]:
import os
import yt_dlp
from moviepy.video.io.VideoFileClip import VideoFileClip

# Function to extract YouTube URLs from an HTML file
def extract_youtube_urls(html_file_path):
    from bs4 import BeautifulSoup

    with open(html_file_path, 'r', encoding='utf-8') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    thumbnails = soup.find_all('a', {'id': 'thumbnail'})
    urls = []

    # Extract unique YouTube URLs
    for thumbnail in thumbnails:
        href = thumbnail.get('href')
        if href and '/watch?v=' in href:
            urls.append("https://www.youtube.com" + href)

    # Remove duplicates
    urls = list(set(urls))
    return urls

# Function to download videos from YouTube using yt-dlp
def download_youtube_videos(urls, download_folder="./videos"):
    # Create folder if it doesn't exist
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    failed = []

    for url in urls:
        try:
            ydl_opts = {
                'outtmpl': os.path.join(download_folder, '%(title)s.%(ext)s'),  # Save to the folder
                'format': 'bestvideo+bestaudio/best',  # Download best quality
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            print(f"Downloaded: {url}")
        except Exception as e:
            print(f"Failed to download {url}: {e}")
            failed.append(url)

    return failed

# Function to convert MP4 files to MP3 (requires moviepy)
def convert_mp4_to_mp3(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    files = os.listdir(input_folder)

    for file in files:
        if file.endswith(".mp4"):
            input_path = os.path.join(input_folder, file)
            output_file = os.path.splitext(file)[0] + ".mp3"
            output_path = os.path.join(output_folder, output_file)

            try:
                video_clip = VideoFileClip(input_path)
                audio_clip = video_clip.audio
                audio_clip.write_audiofile(output_path, codec='mp3')
                print(f"Converted: {file} to MP3")

                # Close clips
                audio_clip.close()
                video_clip.close()
            except Exception as e:
                print(f"Error converting {file}: {e}")

if __name__ == "__main__":
    # Extract YouTube URLs from the HTML file
    html_file = './youtube.html'
    urls = extract_youtube_urls(html_file)

    # Save URLs to file
    with open('urls.txt', 'w') as f:
        for url in urls:
            f.write(url + '\n')

    # Download the YouTube videos
    failed_downloads = download_youtube_videos(urls)

    # Optionally, save failed downloads to a file
    if failed_downloads:
        with open('failed_urls.txt', 'w') as f:
            for url in failed_downloads:
                f.write(url + '\n')

    # Convert downloaded MP4 videos to MP3
    input_folder = "./videos/"
    output_folder = "./mp3/"
    convert_mp4_to_mp3(input_folder, output_folder)


ModuleNotFoundError: No module named 'yt_dlp'